In [1]:
import os
import pandas as pd
from pandas import read_excel
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

def find(name,path):
    
    for root, dirs, files in os.walk(path):        
        if name in files:
            return os.path.join(root, name)
        
#Creating source filepath
def src_filepath(name,path):
        
        src_filepath = find(name, path)             #Source filepath
        print ("Source filepath successfully created")
        return src_filepath

def dest_filepath():
        dest_name = 'Sendai_screen_results.xlsx'
        dest_path = r'D:\Users\INSTR-ADMIN\Dropbox (NYSCF)\NYSCF Team Folder\FACS_Share\ProTeam'
        dest_filepath = find(dest_name, dest_path)             #Source filepath
        print ("Destination filepath successfully created")
        return dest_filepath


def create_sheet(dest_folder,plate_id):
    dest_wb = openpyxl.load_workbook(dest_folder)
    dest_wb.create_sheet(plate_id)
    dest_wb.save(dest_folder)
    dest_wb.close()
    
    
    
#Creating Sample list
def samples_list():
    samples_list = []
    for sample in (df['Sample']):
        if sample not in samples_list:
            samples_list.append(sample)
    return samples_list

#Creating other relevant columns
def count_list():
    count_list = []
    for i in range(0,len(df)):
        if df['Gate'][i] == 'All Events':
            count_list.append(df['Count'][i])
        else:
            pass
    return count_list

#Creating other relevant columns
def gated_list():
    gated_list = []
    for i in range(0,len(df)):
        if df['Gate'][i] == 'TotalSendai_R5':
            gated_list.append(df['%Gated'][i])
        else:
            pass
    return gated_list

#Creating Status column - Pass, Fail, Question?
def status(gated_list,count_list):
    pass_list = []
    for j in range(0,len(gated_list)) :
        if gated_list[j]<7 and count_list[j] > 1000:
            pass_list.append('Pass')
        elif 7 < gated_list[j] < 10:
            pass_list.append('Acceptable')
        else:
            pass_list.append('Fail')
    return pass_list

#Func to highlight background colors based on their status        
def highlight_cells(cell_value): 
       
    if cell_value == 'Pass':
        return 'background-color: #7FFF00'
    elif cell_value == 'Acceptable':
        return 'background-color: #98F5FF'
    elif cell_value == 'Fail':
        return 'background-color: #FF3030'
    else:
        pass

#Creating the file    
def append_sheet():
    #Defining the plate ID by user input
    plate_id= "MCON" + input("Plate ID: ") + "_FLOW_201"
    #Establishing source file location
    src_name = plate_id + '.xlsx'
    src_path = r'C:\Users\bioen\ML\Sendai screening data'
    filepath = src_filepath(src_name, src_path)
    #Setting up destination file location
    dest_folder = dest_filepath()
    #Extracting data from source file
    global df
    df = pd.read_excel(filepath)
    samples = samples_list()
    counted_list = count_list()
    gated_count_list = gated_list()
    final_status = status(gated_count_list,counted_list)
    #Assigning the extracted data to a variable
    extracted_data = pd.DataFrame({'Sample ID': samples,
                          'Event Count': counted_list,
                          'Total sendai': gated_count_list,
                                  'Status': final_status})
    # writing to Excel in .xlsx format
    sendai_screen_results = pd.ExcelWriter(r'D:\Users\INSTR-ADMIN\Dropbox (NYSCF)\NYSCF Team Folder\FACS_Share\ProTeam\Sendai_screen_results.xlsx', engine='openpyxl', mode='a')
    extracted_data.style.applymap(highlight_cells).to_excel(sendai_screen_results, sheet_name=plate_id, index=False)
    # saving the excel file
    sendai_screen_results.save()
    print('Sendai data is successfully written into Excel File') 